In [61]:
import json
import matplotlib.pyplot as plt
import numpy as np
import operator
import pandas as pd
import re
import seaborn as sns


from classifier import *
from heapq import nlargest
from issues import get_num_code_lines
from nltk.stem import PorterStemmer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [62]:
with open('../data/flutter/flutter_issues_labeled.json') as json_data:
    issues = json.load(json_data)
print("Number of issues: " + str(len(issues)))

Number of issues: 7170


In [63]:
with open('../data/flutter/flutter_issues_prs_final_5.json') as json_data:
    issues_prs = json.load(json_data)
print("Number of issues: " + str(len(issues_prs)))

success = []
failed = []
partial = []
for issue in issues_prs:
    if issue['merged_prs'] or issue['master_commits']:
        success.append(issue)
    elif issue['local_commits'] or issue['failed_prs']:
        #print(issue)
        partial.append(issue)
    else:
        failed.append(issue)
print("Number of successful issues: " + str(len(success)))
print("Number of partially successful issues: " + str(len(partial)))
print("Number of failed issues: " + str(len(failed)))

Number of issues: 11146
Number of successful issues: 4059
Number of partially successful issues: 185
Number of failed issues: 6902


In [64]:
#TODO: fix the extra PRs from code #s

In [65]:
issue_map = {}
for issue in issues_prs:
    completed_by = set()
    failed_by = set()
    for pr in issue['merged_prs']:
        if pr['author']:
            completed_by.add(pr['author'].replace('/', ''))
    for commit in issue['master_commits']:
        if commit['author']:
            completed_by.add(commit['author'].replace('/', ''))
    for pr in issue['failed_prs']:
        if pr['author'] and pr['author'] not in completed_by:
            failed_by.add(pr['author'].replace('/', ''))
    for commit in issue['local_commits']:
        if commit['author'] and commit['author'] not in completed_by:
            failed_by.add(commit['author'].replace('/', ''))
    issue_map[issue['issue_num']] = {}
    issue_map[issue['issue_num']]['completed_by'] = completed_by
    issue_map[issue['issue_num']]['failed_by'] = failed_by

In [66]:
# create a data frame from the list of issues
df_list = []
for issue in issues:
        if issue['number'] not in issue_map:
            continue
        df_dict = {}
        df_dict['comments'] = issue['comments']
        if (not issue['body']):
            issue['body'] = ""
        df_dict['title'] = issue['title']
        df_dict['issue_num'] = issue['number']
        df_dict['body'] = issue['body']
        df_dict['closed_date'] = pd.to_datetime(issue['closed_at'])
        df_dict['created_date'] = pd.to_datetime(issue['created_at'])
        df_dict['completed_by'] = issue_map[issue['number']]['completed_by']
        df_dict['failed_by'] = issue_map[issue['number']]['failed_by']
        if len(df_dict['completed_by']) == 0:
            continue
        df_dict['labels'] = [label['name'] for label in issue['labels']]
        df_dict['assignees'] = [assignee['login'] for assignee in issue['assignees']]
        for assignee in df_dict['assignees']:
            if assignee not in df_dict['completed_by'] and assignee not in df_dict['failed_by']:
                df_dict['failed_by'].add(assignee)
        df_dict['completed_by'] = list(df_dict['completed_by'])
        df_dict['failed_by'] = list(df_dict['failed_by'])
        df_list.append(df_dict)
df = pd.DataFrame(df_list).sort_values('closed_date')
df.tail(2)

,assignees,body,closed_date,comments,completed_by,created_date,failed_by,issue_num,labels,title
3383,[jonahwilliams],Turn TalkBack on and double tap on a checkbox ...,2018-05-26 01:42:03,10,[jonahwilliams],2018-05-21 17:36:14,[],17775,"[a: accessibility, from: a11y review, severe: ...",[a11y]: Activating a switch or checkbox with T...
3379,"[Hixie, tvolkert]",Because of the way `testWidgets` modifies the ...,2018-05-26 05:04:15,7,[tvolkert],2018-05-19 03:54:20,"[Hixie, filiph]",17738,"[a: tests, framework]",No way to test periodic Streams in `testWidgets`


In [67]:
success_events = 0
partial_events = 0 # local commit / failed pr / assigned but didnt complete (make sure to filter completed events)

In [68]:
# TODO: exclude most experienced devs

num_correct_k = 0
num_correct_total = 0
num_failed_k = 0
num_failed_total = 0
k = 1
dev_counts = {}
devs = set()
for _, row in df.iterrows():
    for failer in row['failed_by']:
        if dev_counts.get(failer, 0) == (k-1):
                print(row['issue_num'])
                print(failer)
                print()
                num_failed_k += 1
        else:
            num_failed_total += 1
        devs.add(failer)
    for completer in row['completed_by']:
        if dev_counts.get(completer, 0) == (k-1):
                num_correct_k += 1
        else:
            num_correct_total += 1
        devs.add(completer)
        dev_counts[completer] = dev_counts.get(completer, 0) + 1
print("Num correct events for first " + str(k) + " contributor: " + str(num_correct_k)  )
print("Total events for first k contributor: " + str(num_correct_k + num_failed_k))
print()
print("Num correct events for all assignees: " + str(num_correct_total))
print("Total events for all assignees: " + str(num_correct_total+num_failed_total))
print()
print("Total number of unique devs: " + str(len(devs)))

299
krisgiesing

3297
vlidholt

6952
a-siva

5488
filiph

6926
zoechi

7551
aqnouch

7551
Mayho

7551
MrGeislinger

7551
DanTup

7551
nikai3d

7551
xudshen

7551
ssaurel

7551
LyndonChin

7551
Scorpiion

8140
szakarias

8850
mravn-google

8878
aqnouch

8878
Mayho

8878
MrGeislinger

8878
mkustermann

8878
DanTup

8878
nikai3d

8878
LyndonChin

8115
mattsarett

8788
mravn-google

8800
mkustermann

9442
rxlabz

9668
kevmoo

9910
mkustermann

9910
DanTup

10193
aqnouch

10193
Mayho

10193
MrGeislinger

10193
mkustermann

10193
DanTup

10193
nikai3d

10193
xudshen

10193
ssaurel

10193
LyndonChin

10193
Scorpiion

10050
ajonp

7135
aqnouch

7135
Mayho

7135
MrGeislinger

7135
mkustermann

7135
DanTup

7135
nikai3d

7135
xudshen

7135
ssaurel

7135
LyndonChin

7135
Scorpiion

10411
aqnouch

10411
Mayho

10411
MrGeislinger

10411
mkustermann

10411
DanTup

10411
nikai3d

10411
xudshen

10411
ssaurel

10411
LyndonChin

10861
aqnouch

10861
Mayho

10861
MrGeislinger

10861
DanTup

10861
nikai3